In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from pylab import *
from pyntcloud import PyntCloud
from matplotlib.pyplot import figure


INFO - 2021-12-18 22:50:41,576 - utils - NumExpr defaulting to 8 threads.


# Visualize Point Cloud

reads a point cloud model of Venice and visualizes it

In [2]:
print("Load a ply point cloud")
venice = o3d.io.read_point_cloud('/Users/liyuxiao/Downloads/Venice.ply/venice_reproduced.ply')

Load a ply point cloud


In [11]:
print("Print (and visualize) the ply point cloud")
print(venice)
print(np.asarray(venice.points))
o3d.visualization.draw_geometries([venice])

Print (and visualize) the ply point cloud
PointCloud with 4093530 points.
[[-3.21571118e+02 -2.48106492e+03  1.64116309e+00]
 [-3.28938886e+02 -2.48008782e+03  1.29477744e+00]
 [-3.28718691e+02 -2.47819047e+03  1.62962471e+00]
 ...
 [ 1.35218538e+03  1.88591577e+02  9.99854807e+00]
 [-7.04763137e+02  9.39715193e+02  7.02373177e+00]
 [-9.63983102e+02  3.89082728e+02  3.52930193e+00]]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Voxel downsampling
Voxel downsampling uses a regular voxel grid to create a uniformly downsampled point cloud from an input point cloud. It is often used as a pre-processing step for many point cloud processing tasks. The algorithm operates in two steps:
1. Points are bucketed into voxels.
2. Each occupied voxel generates exactly one point by averaging all points inside.

In [4]:
print("Downsample the point cloud with a voxel of 0.04")
venice_down = venice.voxel_down_sample(voxel_size = 0.04)

Downsample the point cloud with a voxel of 0.04


After we downsampled the PointCloud, it should be much easier to visualize the PointCloud

In [4]:
o3d.visualization.draw_geometries([venice_down])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Plane segmentation
Open3D also supports segmententation of geometric primitives from point clouds using RANSAC. To find the plane with the largest support in the point cloud, we can use segment_plane. The method has three arguments: distance_threshold defines the maximum distance a point can have to an estimated plane to be considered an inlier, ransac_n defines the number of points that are randomly sampled to estimate a plane, and num_iterations defines how often a random plane is sampled and verified. The function then returns the plane as (𝑎,𝑏,𝑐,𝑑) such that for each point (𝑥,𝑦,𝑧) on the plane we have 𝑎𝑥+𝑏𝑦+𝑐𝑧+𝑑=0. The function further returns a list of indices of the inlier points.

In [3]:
plane_model, inliers = venice_down.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a}x + {b}y + {c}z + {d} = 0")

Plane equation: 0.00016659863078426673x + 0.0007392040975757255y + 0.9999997129110579z + 0.0807718749686342 = 0


Visualize the plane

In [4]:
inlier_cloud = venice_down.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = venice_down.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Point cloud outlier removal
statistical_outlier_removal removes points that are further away from their neighbors compared to the average for the point cloud. It takes two input parameters:

1. nb_neighbors, which specifies how many neighbors are taken into account in order to calculate the average distance for a given point.

2. std_ratio, which allows setting the threshold level based on the standard deviation of the average distances across the point cloud. The lower this number the more aggressive the filter will be.



In [121]:
print("Statistical oulier removal")
venice_down, ind = venice_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
print("Radius oulier removal")
cl, ind = venice_down.remove_radius_outlier(nb_points=16, radius=0.05)

Statistical oulier removal
Radius oulier removaltliers: [=>                                      ] 2%
Remove statistical outliers: [========================================] 100%
Remove radius outliers: [========================================] 100%


Get the coordinates of the points and the total number of points

In [7]:
venice_coordinate = np.asarray(venice_down.points)
print('coordinates of points:')
print(venice_coordinate)
print('total number of points:')
len(venice_coordinate)

coordinates of points:
[[  0.76118314 -20.92252541  47.1059494 ]
 [  0.63333982 -20.92252541  47.08749771]
 [  0.63729376 -20.90539169  47.05981827]
 ...
 [ 30.0746727   -3.96003485   5.44277859]
 [ -5.58549881   0.5007506   -6.63570881]
 [-10.13751984  -3.01791453   1.95749509]]
total number of points:


4093530

# Point cloud Redress

Please note that here we need to do an iteration so that we could get a more precise model

1. We first rotate the PointCloud so that the plane in the PointCloud is XoY plane
2. Then we scale the PointCloud so that the height in the model would match the height in real world
3. In the end, we translate the PointCloud so that the plane equation could be Z=0

In [5]:
for i in range(100):

    cos = c
    sin = np.sqrt(a**2+b**2)
    u1= b
    u2= -a

    R = [[cos+u1**2*(1-cos), u1*u2*(1-cos), u2*sin],
    [u1*u2*(1-cos),cos+u2**2*(1-cos),-u1*sin],
    [-u2*sin, u1*sin, cos]]
    
    venice_down.rotate(R, center=venice_down.get_center())

    venice_coordinate = np.asarray(venice_down.points)
    distance = np.zeros((len(venice_coordinate),1))
    for i in range(len(venice_coordinate)):
        distance[i] = abs(venice_coordinate[i,0]*a+ venice_coordinate[i,1]*b+ venice_coordinate[i,2]*c +d)/np.sqrt(a**2+b**2+c**2)

    venice_down.scale(98.6/distance[np.argmax(distance, axis=0)][0,0], center=venice_down.get_center())

    plane_model, inliers = venice_down.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
    [a, b, c, d] = plane_model
    venice_down = venice_down.translate((0, 0, d))
    
    print(f"Plane equation: {a}x + {b}y + {c}z + {d} = 0")

Plane equation: 7.746926473970973e-05x + 0.0006669445225433313y + 0.9999997745917331z + 0.0453388697521056 = 0
Plane equation: 0.00010366723649106242x + 0.0009458163277750072y + 0.9999995473421867z + 0.1266336400268746 = 0
Plane equation: -0.000943271509038323x + 0.000991546997642555y + 0.9999990635362672z + -0.6945237757280741 = 0
Plane equation: -0.0005553443634431247x + 0.0009488583777942256y + 0.9999993956300258z + 0.21321948709904837 = 0
Plane equation: 0.00011816087714508158x + 0.000667307430805578y + 0.9999997703693736z + 0.35167620367785396 = 0
Plane equation: 0.00019875935409840924x + 0.0006949919946080666y + 0.999999738740389z + -0.06510987139227913 = 0
Plane equation: 0.00024459938396724983x + 0.0007413940845706907y + 0.99999969525293z + 0.0708162258173986 = 0
Plane equation: 0.00026000908359397534x + 0.0008423892284613589y + 0.9999996113877567z + 0.1042041366285544 = 0
Plane equation: 3.287474215005547e-05x + 0.0008211792314751342y + 0.9999996622919035z + 0.0070404386796446

# Verification

In [5]:
venice_coordinate = np.asarray(venice_down.points)
distance = np.zeros((len(venice_coordinate),1))
for i in range(len(venice_coordinate)):
    distance[i] = abs(venice_coordinate[i,0]*a+ venice_coordinate[i,1]*b+ venice_coordinate[i,2]*c +d)/np.sqrt(a**2+b**2+c**2)

Z coordinate of the height point in Venice: (reference:98.6m)

In [6]:
venice_coordinate[np.argmax(distance, axis=0)][0,2]

98.48128048053749

Distance from the height ppoint to the ground: (reference:98.6m)

In [7]:
distance[np.argmax(distance, axis=0)][0,0]

98.34577633244484

Take a look at the PointCloud model you constructed

In [9]:
o3d.visualization.draw_geometries([venice_down])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Dada! Right now we finished the model construction part, we could save the model here.

In [31]:
o3d.io.write_point_cloud("/Users/liyuxiao/Downloads/Venice/venice_temp.ply", venice_down)

True